In [ ]:
# Initial notebook setup -- RUN ME FIRST!
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'font.size': 14})
rcParams['figure.figsize'] = (10, 5)

# Fitting exoplanet transit models to Kepler light curves

[Kepler](https://www.nasa.gov/mission_pages/kepler/spacecraft/index.html) is a space telescope designed to search for planets around other stars (exoplanets). The primary detection technique used by Kepler is transit photometry. It continuously monitors a field of view containing ~150,000 main sequence stars for more than three years. If an exoplanet transits in front of any of those stars, it will be detected as a periodic dimming. It is important to remember that Kepler will find a **biased** population -- planets with large orbital radius have lower probability to be well-aligned for transits and their orbital periods could be too long for Kepler to observe periodicity.

Kepler data can be accessed through the [Barbara A. Mikulski Archive for Space Telescopes](https://archive.stsci.edu/kepler/data_search/search.php). If you follow that link, it will take you to a search form that lets you query with all sorts of criteria. For this exercise, we are going to focus on [Kepler 8-b](https://archive.stsci.edu/kepler/published_planets/search.php?action=Search&kepler_name=Kepler-8+b) (Kepler ID 6922244). This is an example of a "hot Jupiter". The planet radius that is 14.6 times larger than Earth, but it orbits a Sun-like star at an orbital distance of just 0.05 AU and period of 3.522 days. Click on the Kepler 8-b link above and take a look at the information provided in the table. 

From the Kepler data archive, we can download a file containing the *light curve* for this star. A light curve is simply the brightness (flux) of the star as a function of time. In this case, we are going to download the data for Kepler ID 6922244 from January 5 2012 through March 28 2012.

In [ ]:
# Download Kepler 8-b light curve from archive.stsci.edu
# Only need to run this cell once.
!wget https://archive.stsci.edu/missions/kepler/lightcurves/0069/006922244/kplr006922244-2012088054726_llc.fits


The light curve data is contained in a [FITS file](https://fits.gsfc.nasa.gov/), which is a file format that is widely used in astronomy. We don't need to delve into the details of FITS files because the `astropy` module (python library for astronomy) includes tools for reading data in FITS format. The block of code below will read the light curve file that you just downloaded and print the names of the data fields that it finds in the file.

In [ ]:
# Import FITS read/write tools from astropy module.
from astropy.io import fits

# Open the FITS file containing the light curve for Kepler 8-b 
# and select the data from Header Data Unit (HDU) 1.
k8b = fits.open('kplr006922244-2012088054726_llc.fits')[1].data
# Print names of the data fields.
print(k8b.names)


You can see that this file contains a lot of different data fields. We are going to ignore most of these, except for the following important ones:
* `k8b['TIME']` = time value for each sample, in units of days starting from noon on January 1 2009.
* `k8b['SAP_FLUX']` = measured flux of the star for each time sample, in raw CCD units (electrons / second).
* `k8b['PDCSAP_FLUX']` = measured flux of the star after applying corrections to remove long term instrumental drifts and other similar effects.
* `k8b['PDCSAP_FLUX_ERR']` = statistic uncertainty on the corrected flux.

Notice the syntax for selecting data fields&mdash;you need to provide the exact field name (case-sensitive) as a string (in quotes) inside square brackets.

The first thing to do is to take a look at the data. Plot both the uncorrected (`'SAP_FLUX'`) and corrected (`'PDCSAP_FLUX'`) flux as a function of time. You should be able to easily pick out the periodic exoplanet transits. Note that there are some gaps in the data (with more gaps in the corrected flux data). These gaps show up in the data array as `NaN` values (i.e. not a number). They can be due to instabilities in the instrument or cosmic ray hits on the focal plane CCDs.

You can try plotting other fields if you like. For a full description of all data fields, you can read Section 2.3.1 of the [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf). (I'm not actually suggesting that you should do this now.)


In [ ]:
# Plot both uncorrected and corrected flux as a function of time.


Next, we want to fold the light curve so that all of the transits stack up on top of each other. To do this, we can convert our `TIME` data array into phase by dividing by the orbital period (3.522 days) and using the [modulo operation](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mod.html) to translate the phase onto the range [0,1]. Because of some subtle details in how the `astropy.io.fits` module works, we will first copy the data to some new variables.

If you try calculating the phase using the wrong period, you will find that the transit samples do not line up neatly with one another. Note that you can apply an arbitrary offset to the phase, which will just modify where the transit occurs. This is not important for our purposes, but it will be helpful for later on to make sure that the transit doesn't occur near the boundary where phase wraps around from 1 to 0.


In [ ]:
# Copy light curve data arrays.
time = k8b['TIME'].copy()
flux = k8b['PDCSAP_FLUX'].copy()
flux_err = k8b['PDCSAP_FLUX_ERR'].copy()

# Now, calculate phase from time.
# Divide time by orbital period (tau) and use the np.mod function to determine the result modulo 1.
tau = 3.522 # Orbital period, in days.
phase =     ######### CALCULATE PHASE HERE #########

# Plot the folded light curve.
plt.plot(phase, flux, '.')


## Very simple transit model

We are going to start very simple for our first exoplanet model. In this model, the planet is either eclipsing the star or it is not (no transition region). When the eclipse is occurring, the observed flux will be reduced by fractional amount

$$
\begin{equation}
\frac{\Delta f}{f} = \left( \frac{r_p}{r_*} \right)^2 ,
\end{equation}
$$

where $r_p$ is the radius of the planet and $r_*$ is the radius of the star.

The duration of the eclipse, $t_1$, is the time that it takes for the planet to travel a distance equal to the diameter of the star, i.e. $2 r_*$. We have already converted our time coordinate into orbital phase, so we want to calculate this time in units of the orbital period.

$$
\begin{equation}
t_1 = \frac{2 r_*}{v}
\end{equation}
$$

But we can rewrite the orbital velocity of the planet, $v$, in terms of orbital period, $\tau$, and orbital radius, $R$.

$$
\begin{align}
v &= \frac{2 \pi R}{\tau} \\
\frac{t_1}{\tau} &= \frac{r_*}{\pi R}
\end{align}
$$

Now we have all the pieces we need to write a function that calculates this model. Our function will take two inputs: an array of model parameters and an array of phase values. Using the parameters, we will calculate the expected flux for each phase value provided, so the result returned by this function will be an array of fluxes with the same size as the input array of phases. 

We are packing the model parameters into a single array so that we can use this function with a minimization algorithm later. I suggest using the following parameters (though there are other valid choices and you can change this if you like):
* `param[0]` = baseline flux when eclipse is not occurring
* `param[1]` = phase value when the eclipse begins, *i.e.* $\phi_0$
* `param[2]` = radius of planet / radius of orbit, *i.e.* $r_p / R$
* `param[3]` = radius of star / radius of orbit, *i.e.* $r_* / R$

To write this function, you will need to figure out which samples have phase that falls during the eclipse. One way to do this would be to write a `for` loop that examines every phase value and checks whether is has phase in that range. But a *much better* way of doing this is to use functions that test every value in the array at once. numpy arrays naturally support logical (Boolean) operations on the entire array and you can use the [`np.where`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html) function to identify the indices of samples for which the operation evaluates to `True`.

For example, if `ph0` is the phase at the start of the eclipse and `ph1` is the phase at the end of the eclipse, you can do the following:
```
# Find indices where phase is between ph0 and ph1.
idx = np.where((phase >= ph0) & (phase < ph1))
# Specify that flux should be 1 - df for those particular samples.
# This assumes that flux and phase are arrays with matching length!
flux[idx] = 1.0 - df
```


In [ ]:
def model0(param, phase):
    """
    Calculate simple exoplanet transit model.
    
    Inputs
    ------
    param : array, shape=(4,)
        Array of model parameters, ordered as above.
    phase : array, shape=(N,)
        Array of phase values in range [0,1] for which to calculate flux.
        
    Returns
    -------
    flux : array, shape=(N,)
        Array of flux values with same shape as input phase array.
    
    """
    
    # Unpack model parameters.
    f0 = param[0]  # baseline flux
    ph0 = param[1] # phase at start of eclipse
    rp = param[2]  # radius of planet
    rs = param[3]  # radius of star
    
    # Calculate flux.
    # Start with an array of ones that has the same size as the input phase array.
    flux = np.ones(phase.shape)

    # Now you need to find the phase values between ph0 and ph0 + t1/tau (i.e. samples during the eclipse) 
    # and modify the flux for those values so that it is decreased by factor df/f.
    ############# ADD CODE HERE ################
    
    # So far we have a flux array that is equal to one when the planet is not transiting,
    # then decreases fractionally when the planet is transiting. We want to compare this 
    # to actual flux data, so we need to multiply by our baseline flux to normalize.
    flux = flux * f0
    
    # Done -- return the result.
    return flux
    

Try using this function to model the light curve data. Look at your plot of flux vs phase above and try to guess reasonable values for the model parameters. It should be fairly straightforward to estimate the baseline flux and the phase at the start of transit, but the radius of the planet and the star are less obvious. If you plug in your guesses below, you can use that code to evaluate the model and plot it on top of the data.


In [ ]:
###### GUESS MODEL PARAMETERS HERE ######
f0 =     # baseline flux
ph0 =    # phase at start of eclipse
rp =     # radius of planet / orbital radius (probably something like 0.01)
rs =     # radius of star / orbital radius (probably something like 0.1)
# Pack these parameters into an array.
param = np.array([f0, ph0, rp, rs])
# Plot the light curve data.
plt.plot(phase, flux, '.', label='data')
# Evaluate the model for phases from 0 to 1.
x = np.arange(0, 1, 0.001)
y = model0(param, x)
# Plot the model.
plt.plot(x, y, 'r', label='model')
plt.legend()
# It might be helpful to zoom in on the eclipse region, using the plt.xlim function.


If you play around with your guesses long enough, you might be able to get something that matches the light curve reasonably well. However, the computer can do a much better job of exploring all of the options and finding the best one. To do this, we need to define a statistic that quantifies how well the model agrees with the data. The most common statistic for this is $\chi^2$ ("chi-squared")
$$
\begin{equation}
\chi^2 = \sum_i \frac{(m_i - d_i)^2}{\sigma_i^2}
\end{equation}
$$
where $m_i$ are the model values, $d_i$ are the data values, and $\sigma_i$ are the statistical uncertainties for each data value.

Write a simple function to calculate $\chi^2$.


In [ ]:
def chi2(model, data, err):
    """
    Calculates chi^2.
    
    Inputs
    ------
    model : array, shape=(N,)
        Array of model values.
    data : array, shape=(N,)
        Array of data values.
    err : array, shape=(N,)
        Array of statistical uncertainties for each data value.
        
    Returns
    -------
    chi2 : float
        Calculated chi^2 value.
    
    """
    
    ####### Write your function here. 
    # Don't forget the summation! Use the np.nansum function to avoid running into trouble with
    # missing values in the Kepler data.
    
    return chi2
    

The `scipy` (scientific python) module includes a variety of [minimization algorithms](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize). The way that these algorithms work is that you give them a function that takes the array of parameters as an input and returns some statistic (like $\chi^2$). Starting from some initial guess, the algorithm will play around with the parameters until it has found the combination that minimizes the statistic. Different algorithms have different strengths and weaknesses&mdash;we will use ["Nelder-Mead"](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-neldermead.html#optimize-minimize-neldermead), which is generally quite reliable.


In [ ]:
# Need to import minimization algorithms from scipy.
from scipy.optimize import minimize

The `chi2` function that you wrote above isn't exactly what we want, because its inputs include the evaluated model, the data, and the errors. We are going to use a neat python feature known as [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to quickly define a function that conforms to what we need for the minimization algorithm. The syntax for a lambda expression is a little bit odd:
```
new_function = lambda p: old_function(p, x1, x2, x3)
```
What this means is that we just created a function called `new_function`. This function takes only one argument, `p`. When you call it with that argument, it will evaluate `old_function(p, x1, x2, x3)` where `x1`, `x2`, and `x3` were some variables that were already defined at the time that we created `new_function`.

Let's put this into action below. The notation is very compact, but you can see that when we call `minfunc`, it will evaluate `model0` using parameter list `p` and our phase values, then it calculates $\chi^2$ for that model, the data values, and the data uncertainties.

When you 


In [ ]:
# Use a lambda expression to create an function to use for our minimizer.
# Remember that the minimization function should take the parameter list as an input argument
# and it should return chi^2.
minfunc = lambda p: chi2(model0(p, phase), flux, flux_err)

# Need to supply an initial guess for the parameters. The result shouldn't really 
# depend on your guess, but it helps if the guess isn't too far off.
f0 =     # baseline flux
ph0 =    # phase at start of eclipse
rp =     # radius of planet / orbital radius (probably something like 0.01)
rs =     # radius of star / orbital radius (probably something like 0.1)
guess = np.array([f0, ph0, rp, rs])

# Now we can run our minimization algorithm.
res = minimize(minfunc, guess, method='Nelder-Mead')
print(res)


The minimization function returns a data structure that contains a lot of information. The most important part are the best-fit parameters that it found, which are stored in `res.x`. The value of $\chi^2$ for that best-fit model is stored in `res.fun`. It also contains some status flags that tell you whether the algorithm terminated successfully (*i.e.* whether it found a minimum that satisfied its criteria). 

Plot the best-fit model on top of the data. How does this compare to your attempt at fitting the light curve by hand?


In [ ]:
# Plot the light curve data.
plt.plot(phase, flux, '.', label='data')
# Evaluate the model for phases from 0 to 1.
x = np.arange(0, 1, 0.001)
y = model0(res.x, x)
# Plot the model.
plt.plot(x, y, 'r', label='model')
plt.legend()
# It might be helpful to zoom in on the eclipse region, using the plt.xlim function.


The [database entry for Kepler 8-b](https://archive.stsci.edu/kepler/published_planets/search.php?action=Search&kepler_name=Kepler-8+b) says that the planet's radius is 14.59 times Earth, the ratio of planet radius to stellar radius is 0.092, and the orbital radius is 0.047 AU. How do these compare to the best-fit values you found?

From the plot that compares model and data, it is obvious that our simple model does not fully capture the details of the light curve. Think of some extensions to this model that might improve the fit.
